In [6]:
import pandas as pd
import numpy as np
import requests
import json
import hvplot.pandas
from dotenv import load_dotenv
from datetime import date
import os
from scipy import stats
from numpy.lib.stride_tricks import as_strided
from numpy.lib import pad
#import pad
import matplotlib.pyplot as plt
%matplotlib inline

from pathlib import Path

In [7]:
## Load pickle for exports and imports of data  
import pickle 
def load_obj(path):
    with open(path, 'rb') as f:
        return pickle.load(f)
    
def save_obj(obj, path ):
    with open(path, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

In [3]:
#load_dotenv()

In [103]:
class TechnicalIndicators:
    
    def __init__(self, stock_list):
        
#         self.ticker = ticker
        self.stock_list = stock_list
    
    def rolling_spearman(self, seqa, seqb, window):
        stridea = seqa.values.strides[0]
        ssa = as_strided(seqa, shape=[len(seqa) - window + 1, window], strides=[stridea, stridea])
        strideb = seqa.values.strides[0]
        ssb = as_strided(seqb, shape=[len(seqb) - window + 1, window], strides =[strideb, strideb])
        ar = pd.DataFrame(ssa)
        br = pd.DataFrame(ssb)
        ar = ar.rank(1)
        br = br.rank(1)
        corrs = ar.corrwith(br, 1)
        return pad(corrs, (window - 1, 0), 'constant', constant_values=np.nan)
    
    def create_price_df(self,ticker, period='daily'):
        
        if period =='hourly':
            api_url = 'https://fmpcloud.io/api/v3/historical-chart/1hour'
        else:
            api_url = 'https://fmpcloud.io/api/v3/historical-price-full'
            
        ticker_df = json.loads(requests.get(f"{api_url}/{ticker}?apikey={fmp_api}").content)['historical']
        data = pd.DataFrame(ticker_df).set_index('date')[::-1]
        data['Date'] = data.index
        data.index = data.index.astype('datetime64[ns]')
        
        return data
    
    def use_csvs(self, ticker):
    
        #data = pd.read_csv("../FilesExportIndividualStockDFs_Big/"+ticker+"_combined_df.csv", index_col='Date', parse_dates=True)
        path = Path('../FilesExport_Finished/'+ticker+'_finished_df.pkl')
        data_import = load_obj(path)
        data = data_import['dataFrame'].copy()
        
        return data
        
        
        
    def bollinger_bands(self, dataframe,period=20):
        data = dataframe.copy()
        data['middle_band'] = data[['adjClose']].rolling(window=period).mean()
        data[str(period)+'_day_stdev'] = data[['adjClose']].rolling(window=period).std()
        data['upper_band'] = data['middle_band']+2*data[str(period)+'_day_stdev']
        data['lower_band'] = data['middle_band'] - 2*data[str(period)+'_day_stdev']
        data['spread'] = data['upper_band'] + data['lower_band']
        data['change_in_spread'] = data['spread']/data['spread'].shift(1)-1
        data[str(period)+"_return"] = data['adjClose']/data['adjClose'].shift(period)-1
        data['bollinger_signal'] = data['change_in_spread'].rank(ascending=False, pct=True)
        data.dropna()
        
        return data
    
    def dema(self, dataframe, period1=10, period2=20):
        data = dataframe.copy()
        data[str(period1)+'ema1'] = dataframe[['adjClose']].ewm(span=period1, adjust=False).mean()
        data[str(period1)+'ema2'] = data[str(period1)+'ema1'].ewm(span=period1, adjust=False).mean()
        data[str(period1)+'dema'] = 2*data[str(period1)+'ema1'] - data[str(period1)+'ema2']
        data[str(period2)+'ema1'] = data[['adjClose']].ewm(span=period2, adjust=False).mean()
        data[str(period2)+'ema2'] = data[str(period2)+'ema1'].ewm(span=period2, adjust=False).mean()
        data[str(period2)+'dema'] = 2*data[str(period2)+'ema1'] - data[str(period2)+'ema2']
        data[str(period1)+"_return"] = data['adjClose']/data['adjClose'].shift(period1)-1
        data['spread'] = data[str(period1)+'dema'] - data[str(period2)+'dema']
        data['dema_signal'] = data['spread'].rank(ascending=True, pct=True)
        data = data.dropna()
        return data
    
    def price_momentum(self, dataframe, smoothing1=0.0571, smoothing2=0.1, periods1=15, periods2=10):
        data = dataframe.copy()
        data['smoothing_factor'] = smoothing1
        data[str(periods1)+"average"] = data['changeOverTime'].rolling(window=periods1).mean()
        smoothing_factor_list = [data.iloc[periods1][str(periods1)+"average"]]
        data = data.dropna()
        i=1
        j=0
        while i < len(data[str(periods1)+"average"]):
            smoothing_factor = data.iloc[i]['changeOverTime']*data.iloc[i]['smoothing_factor'] + smoothing_factor_list[j]*(1-data.iloc[i]['smoothing_factor'])
            smoothing_factor_list.append(smoothing_factor)
            j+=1
            i+=1
        data['35d_custom_smoothing'] = smoothing_factor_list
        data['35d_custom_10'] = data['35d_custom_smoothing']*10
        data['smoothing_factor2'] = smoothing2
        data[str(periods2)+"average"] = data['35d_custom_10'].rolling(window=periods2).mean()
        data = data.dropna()
        smoothing_factor_list2 = [data.iloc[0][str(periods2)+"average"]]
        i=1
        j=0
        while i < len(data[str(periods2)+"average"]):
            smoothing_factor = (data.iloc[i]['35d_custom_10'] - smoothing_factor_list2[j])*data.iloc[i]['smoothing_factor2'] + smoothing_factor_list2[j]
            smoothing_factor_list2.append(smoothing_factor)
            j+=1
            i+=1
        data[str(periods2)+'d_custom_smoothing'] = smoothing_factor_list2
        data[str(periods2)+"_return"] = data['adjClose']/data['adjClose'].shift(periods2)-1
#         data['signal'] = np.where(data[str(periods2)+'d_custom_smoothing'] > data[str(periods2)+'d_custom_smoothing'].shift(1), 1.0, 0.0)
#         data = data.rename(columns={'signal':'price_mo'})
        return data
    
    def get_ichimoku_cloud(self, dataframe, period1=4, period2=8, period3=15):
        
        #TODO generate signal, ichimoku works better in current market regime with shorter periods, being able to respond faster to events than a traditional version
        # The conversion crossing the base would be the signal
        
        data = dataframe.copy()
        data['conversion_line'] = data[['adjClose']].rolling(window=period1).mean()
        data['base_line'] = data[['adjClose']].rolling(window=period2).mean()
        data['senkou_spanA_line'] = (data['conversion_line']+data['base_line'])/2
        data['senkou_spanB_line'] = data[['adjClose']].rolling(window=period3).mean()
        data['lagging_span'] = data['adjClose'].shift(period2)
        data = data.dropna()
        
        return data
    
    def accumulation_distribution_line(self, dataframe):
        
        ##TODO define periodicity and pass as arguments, use the mean as the signal generator, -1 is buy and and 1 is sell
        ##TODO need to add ability to ignore a -1 during a range of 1s
        
        data = dataframe.copy()
        data['money_flow_mult'] = round(((data['adjClose'] - data['low']) - (data['high'] - data['adjClose']))/(data['high'] - data['low']),2)
        data = data.dropna()
        data['money_flow_volume'] = data['money_flow_mult']*data['volume']
        money_flow_multiplier_list = list(data['money_flow_volume'].values)
        adl = [money_flow_multiplier_list[0]]
        i = 1
        while i < len(money_flow_multiplier_list):
            a_d_indicator = adl[i-1]+money_flow_multiplier_list[i]
            adl.append(a_d_indicator)
            i+=1
        data['adl'] = adl
        data['adl_change'] = data['adl']/data['adl'].shift(1)-1
        negative_change_count = [0]*9
        i = 0
        counter = 0
        while i < len(data)-9:
            j=0
            while j < 9:
                if data.iloc[j+i]['adl_change'] <0:
                    counter+=1
                if j %19 == 0:
                    negative_change_count.append(counter)
                    counter = 0
                j+=1
            i+=1
        data['negative_change_counter'] = negative_change_count
        data['9_day_return'] = data['adjClose']/data['adjClose'].shift(9)-1
        data['adl_signal'] = self.rolling_spearman(data['adl'], data['9_day_return'], 9)

        return data

    def rsi(self, dataframe, periods=14):
        data = dataframe.copy()
        data['gains'] = np.where(data['changeOverTime']>0, data['changeOverTime'], 0)
        data['losses'] = np.where(data['changeOverTime']<0, np.absolute(data['changeOverTime']), 0)
        data['average_gain'] = data['gains'].rolling(window=periods).mean()
        data['average_loss'] = data['losses'].rolling(window=periods).mean()
        data['rs'] = data['average_gain']/data['average_loss']
        data['rsi'] = (100 - 100/(1+data['rs']))
        data['rsi_signal'] = data['rsi'].rank(ascending=True, pct=True)
        
        return data
    
    
    def get_ratings(self):
        
        ratings = json.loads(requests.get(f"https://fmpcloud.io/api/v3/historical-rating/{self.ticker}?limit=100&apikey={fmp_api}").content)
        ratings_df = pd.DataFrame(ratings)
        ratings_df['average_rating'] = (ratings_df['ratingScore']+ratings_df['ratingDetailsDCFScore']+ratings_df['ratingDetailsROEScore']+ratings_df['ratingDetailsROAScore'] \
                                        +ratings_df['ratingDetailsDEScore'] + ratings_df['ratingDetailsPEScore']+ ratings_df['ratingDetailsPBScore'])/7
        
        return ratings_df
        
    def get_stock_market_performances(self, dataframe):
        limit = len(dataframe)
        data = json.loads(requests.get(f"https://fmpcloud.io/api/v3/historical-sectors-performance?limit=525&apikey={fmp_api}").content)
        sector_df = pd.DataFrame(data).set_index('date')
        sector_df_clean = sector_df[::-1]
        s_p500 = json.loads(requests.get(f"https://fmpcloud.io/api/v3/historical-price-full/^SP500TR?from="+sector_df_clean.index[0]+"&to="+sector_df_clean.index[-1]+"&apikey="+fmp_api).content)['historical']
        sp_df = pd.DataFrame(s_p500)
        sp_df_clean = sp_df[::-1]
        
        return sp_df_clean

    def get_all_indicators(self, ticker):
        dataframe = self.use_csvs(ticker)
        bb = self.bollinger_bands(dataframe)
        dema = self.dema(dataframe)
        adl = self.accumulation_distribution_line(dataframe)
        rsi = self.rsi(dataframe)
        
        dataframe['bollinger_signal'] = bb['bollinger_signal']
        dataframe['dema_signal'] = dema['dema_signal']
        dataframe['adl_signal'] = adl['adl_signal']
        dataframe['rsi_signal'] = rsi['rsi_signal']
        return dataframe
    
    def merge_data(self):
        ticker_list = self.stock_list
        for stock in ticker_list:
            stock_csv = self.use_csvs(stock)
            stock_csv['Date'] = stock_csv.index.astype("string")
            stock_csv = stock_csv.iloc[:-1]
            stock_csv.index.names =[""]
            stock_indicators = self.get_all_indicators(stock)
            stock_signals = stock_indicators[['Date','bollinger_signal','dema_signal', 'adl_signal', 'rsi_signal']]
            stock_signals.loc[:,'Date'] = stock_signals['Date'].astype('string')
            stock_signals.index.names =[""]
            merged_data = pd.merge(left = stock_csv, right=stock_signals, on=['Date']).set_index('Date')
            merged_data = merged_data.dropna()
            writer = pd.ExcelWriter("NewCsvs/"+stock+"_all_indicator_dfs.xlsx", engine="xlsxwriter")

            merged_data.to_excel(writer, sheet_name=stock+"_data")

            writer.save()
        
        return print("All files successfully saved")

In [17]:
path = Path('../Resources/ftd_key_list_sorted.pkl')
ftd_key_list = load_obj(path)
path = Path('../Resources/etf_key_list_sorted.pkl')
etf_key_list = load_obj(path)
path = Path('../Resources/eqt_key_list_sorted.pkl')
eqt_key_list = load_obj(path)

In [30]:
len(ftd_key_list)

3695

In [18]:
ftd_key_list[0]

'AAA'

In [19]:
## Create short test list 
test_list = eqt_key_list[0:10]

In [20]:
## Set stock_list to be used
stock_list = test_list

In [105]:
stock_technicals = TechnicalIndicators(stock_list)

In [50]:
test_df = stock_technicals.use_csvs('AAPL')
test_df

,open,high,low,close,adjClose,volume,unadjustedVolume,change,changePercent,vwap,changeOverTime,QUANTITY_FAILS,ShortVolumeNSDQ,ShortExemptVolumeNSDQ,TotalVolumeNSDQ,ShortVolumeNYSE,ShortExemptVolumeNYSE,TotalVolumeNYSE
Date,,,,,,,,,,,,,,,,,,
2016-01-04,25.652500,26.342501,25.500000,26.337500,24.251429,270597600.0,270597600.0,0.68500,2.670,26.06000,0.02670,22192.0,7836303.0,91287.0,18109841.0,864417.0,3600.0,2313396.0
2016-01-05,26.437500,26.462500,25.602501,25.677500,23.643715,223164000.0,223164000.0,-0.76000,-2.875,25.91417,-0.02875,2832.0,8716993.0,122578.0,19830715.0,727845.0,500.0,2824607.0
2016-01-06,25.139999,25.592501,24.967501,25.174999,23.181011,273829600.0,273829600.0,0.03500,0.139,25.24500,0.00139,231.0,8571299.0,197769.0,25127700.0,931705.0,400.0,3416895.0
2016-01-07,24.670000,25.032499,24.107500,24.112499,22.202663,324377600.0,324377600.0,-0.55750,-2.260,24.41750,-0.02260,2262.0,7849239.0,90024.0,28064570.0,1055425.0,100.0,4718503.0
2016-01-08,24.637501,24.777500,24.190001,24.240000,22.320068,283192000.0,283192000.0,-0.39750,-1.613,24.40250,-0.01613,1309.0,9894661.0,71580.0,24359618.0,965185.0,0.0,3092569.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-10-25,148.679993,149.369995,147.619995,148.639999,148.423386,50720600.0,50720600.0,-0.03999,-0.027,148.54333,-0.00027,16326.0,4559284.0,54935.0,15100307.0,2372182.0,3947.0,5482668.0
2021-10-26,149.330002,150.839996,149.009995,149.320007,149.102402,60893400.0,60893400.0,-0.01000,-0.007,149.72333,-0.00007,20355.0,5849545.0,43193.0,18466804.0,2065455.0,3963.0,5642089.0
2021-10-27,149.360001,149.729996,148.490005,148.850006,148.633087,56094900.0,56094900.0,-0.51000,-0.341,149.02334,-0.00341,3941.0,5158480.0,63993.0,16397172.0,2563407.0,3828.0,5121334.0


In [102]:
test_df2 = stock_technicals.accumulation_distribution_line(test_df)
test_df2

,open,high,low,close,adjClose,volume,unadjustedVolume,change,changePercent,vwap,...,ShortVolumeNYSE,ShortExemptVolumeNYSE,TotalVolumeNYSE,money_flow_mult,money_flow_volume,adl,adl_change,negative_change_counter,9_day_return,adl_signal
Date,,,,,,,,,,,,,,,,,,,,,
2016-01-04,25.652500,26.342501,25.500000,26.337500,24.251429,270597600.0,270597600.0,0.68500,2.670,26.06000,...,864417.0,3600.0,2313396.0,-3.96,-1.071566e+09,-1.071566e+09,NaN,0,NaN,NaN
2016-01-05,26.437500,26.462500,25.602501,25.677500,23.643715,223164000.0,223164000.0,-0.76000,-2.875,25.91417,...,727845.0,500.0,2824607.0,-5.56,-1.240792e+09,-2.312358e+09,1.157923,0,NaN,NaN
2016-01-06,25.139999,25.592501,24.967501,25.174999,23.181011,273829600.0,273829600.0,0.03500,0.139,25.24500,...,931705.0,400.0,3416895.0,-6.72,-1.840135e+09,-4.152493e+09,0.795783,0,NaN,NaN
2016-01-07,24.670000,25.032499,24.107500,24.112499,22.202663,324377600.0,324377600.0,-0.55750,-2.260,24.41750,...,1055425.0,100.0,4718503.0,-5.12,-1.660813e+09,-5.813307e+09,0.399956,0,NaN,NaN
2016-01-08,24.637501,24.777500,24.190001,24.240000,22.320068,283192000.0,283192000.0,-0.39750,-1.613,24.40250,...,965185.0,0.0,3092569.0,-7.37,-2.087125e+09,-7.900432e+09,0.359025,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-10-25,148.679993,149.369995,147.619995,148.639999,148.423386,50720600.0,50720600.0,-0.03999,-0.027,148.54333,...,2372182.0,3947.0,5482668.0,-0.08,-4.057648e+06,-8.298253e+11,0.000005,7,0.050385,0.766667
2021-10-26,149.330002,150.839996,149.009995,149.320007,149.102402,60893400.0,60893400.0,-0.01000,-0.007,149.72333,...,2065455.0,3963.0,5642089.0,-0.90,-5.480406e+07,-8.298801e+11,0.000066,8,0.059684,0.400000
2021-10-27,149.360001,149.729996,148.490005,148.850006,148.633087,56094900.0,56094900.0,-0.51000,-0.341,149.02334,...,2563407.0,3828.0,5121334.0,-0.77,-4.319307e+07,-8.299233e+11,0.000052,7,0.035406,0.400000


In [113]:
test_all = stock_technicals.get_all_indicators('AAPL')

In [116]:
test_all.head(30)

,open,high,low,close,adjClose,volume,unadjustedVolume,change,changePercent,vwap,...,ShortVolumeNSDQ,ShortExemptVolumeNSDQ,TotalVolumeNSDQ,ShortVolumeNYSE,ShortExemptVolumeNYSE,TotalVolumeNYSE,bollinger_signal,dema_signal,adl_signal,rsi_signal
Date,,,,,,,,,,,,,,,,,,,,,
2016-01-04,25.652500,26.342501,25.500000,26.337500,24.251429,270597600.0,270597600.0,0.6850,2.670,26.06000,...,7836303.0,91287.0,18109841.0,864417.0,3600.0,2313396.0,NaN,NaN,NaN,NaN
2016-01-05,26.437500,26.462500,25.602501,25.677500,23.643715,223164000.0,223164000.0,-0.7600,-2.875,25.91417,...,8716993.0,122578.0,19830715.0,727845.0,500.0,2824607.0,NaN,NaN,NaN,NaN
2016-01-06,25.139999,25.592501,24.967501,25.174999,23.181011,273829600.0,273829600.0,0.0350,0.139,25.24500,...,8571299.0,197769.0,25127700.0,931705.0,400.0,3416895.0,NaN,NaN,NaN,NaN
2016-01-07,24.670000,25.032499,24.107500,24.112499,22.202663,324377600.0,324377600.0,-0.5575,-2.260,24.41750,...,7849239.0,90024.0,28064570.0,1055425.0,100.0,4718503.0,NaN,NaN,NaN,NaN
2016-01-08,24.637501,24.777500,24.190001,24.240000,22.320068,283192000.0,283192000.0,-0.3975,-1.613,24.40250,...,9894661.0,71580.0,24359618.0,965185.0,0.0,3092569.0,NaN,NaN,NaN,NaN
2016-01-11,24.742500,24.764999,24.334999,24.632500,22.681482,198957600.0,198957600.0,-0.1100,-0.445,24.57750,...,9727461.0,88478.0,17236991.0,684897.0,2100.0,2119188.0,NaN,NaN,NaN,NaN
2016-01-12,25.137501,25.172501,24.709999,24.990000,23.010662,196616800.0,196616800.0,-0.1475,-0.587,24.95750,...,10034849.0,88059.0,16543190.0,748171.0,300.0,1873788.0,NaN,NaN,NaN,NaN
2016-01-13,25.080000,25.297501,24.325001,24.347500,22.419056,249758400.0,249758400.0,-0.7325,-2.921,24.65667,...,9669212.0,86945.0,19330706.0,1305540.0,0.0,3444487.0,NaN,NaN,NaN,NaN
2016-01-14,24.490000,25.120001,23.934999,24.879999,22.909374,252680400.0,252680400.0,0.3900,1.592,24.64500,...,10109776.0,150945.0,21082888.0,965611.0,0.0,2862830.0,NaN,NaN,NaN,NaN
